In [ ]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns


In [12]:
filepath_enc = 'data/encounter-events.csv'
filepath_loit = 'data/loitering-events.csv'
filepath_tran = 'data/transshipment-vessels.csv'

In [15]:
df_enc = pd.read_csv(filepath_enc)
df_loit = pd.read_csv(filepath_loit)
df_tran = pd.read_csv(filepath_tran)

In [17]:
print('encounter-events dataframe:')
# print(df_enc.head())  # look at first five instance
print(df_enc.describe())  # get some insight in the data distribution
print(df_enc.info())  # get some insight about data quality
print('All columns: ', df_enc.columns)  # print column names
print('Shape: ', df_enc.shape)  # get shape of dataframe

       fishing_vessel_mmsi  transshipment_vessel_mmsi  mean_latitude  \
count         1.168100e+04               1.168100e+04   11681.000000   
mean          3.492475e+08               3.643690e+08      28.148186   
std           8.560155e+07               1.150885e+08      39.851397   
min           2.000080e+06               2.106310e+08     -64.419519   
25%           2.738125e+08               2.733859e+08      -8.003784   
50%           3.382500e+08               3.415370e+08      50.804512   
75%           4.160038e+08               4.124402e+08      58.732077   
max           9.990001e+08               7.600002e+08      78.223238   

       mean_longitude   duration_hr  median_distance_km  median_speed_knots  
count    11681.000000  11681.000000        11681.000000        11681.000000  
mean        33.759551     13.256271            0.059049            0.603595  
std        127.808306     27.669774            0.068599            0.566881  
min       -179.988139      2.000000    